Primer Job Formater

In [594]:
from MRE import Job
inputDir = "./jugadores/"
promedioDir = "../punto2/contabilizado/"
outputDir = "./format/"

In [595]:
def fmap(key, value, context):
    lista = value.split()
    context.write((int(lista[0]),'RETADO'), int(key))
def fmap2(key, value, context):
    lista = value.split()
    context.write((int(key), 'PP')  , ((float(lista[0]) + 1)/(float(lista[1])+1)))
def cmpShort(key1,key2):
    if(key1[0] == key2[0]):
        return 0
    elif(key1[0] < key2[0]):
        return -1
    else:
        return 1

def cmpShuffle(key1, key2):
    if(key1[1] == key2[1]):
        return 0
    elif(key1[1] == "PP"): #PP es el primero 
        return -1
    else:
        return 1 #Caso retado

def fred(key, values, context):
    acumular_retados = []
    pp = next(values)  
    for v in values:
        acumular_retados.append(v)
    if(len(acumular_retados) == 0):
        acumular_retados = {}
    context.write(key[0], (pp,acumular_retados,puntaje_heroico_inicial))

    
puntaje_heroico_inicial = 1

jobCount = Job(inputDir, outputDir, fmap, fred)
jobCount.setSortCmp(cmpShuffle)
jobCount.setShuffleCmp(cmpShort)
jobCount.setParams(puntaje_heroico_inicial)
jobCount.addInputPath(promedioDir,fmap2)

success = jobCount.waitForCompletion()

print(success)

True


Segundo job iterativo

In [596]:
inputDir = "./format/"
outputDir = "./format/"

In [597]:
def fmap(key, value, context):
    lista = value.split("\t")
    context.write((key,"ESTRUCTURA"),lista)
    set_limpio = lista[1].strip("{} ").split(",")
    if set_limpio[0] != '':
        for v in set_limpio:
            v = v.strip()
            context.write((v, "CONTRIBUCION"), (lista[0],lista[-1]))

def cmpShort(key1,key2):
    if(key1[0] == key2[0]):
        return 0
    elif(key1[0] < key2[0]):
        return -1
    else:
        return 1
    
def cmpShuffle(key1, key2):
    if(key1[1] == key2[1]):
        return 0
    elif(key1[1] == "ESTRUCTURA"): #PP y PH es el primero 
        return -1
    else:
        return 1 #Caso Contribucion

def fred(key, values, context):
    valor = next(values)
    PPi = float(valor[0])
    acumulador=0
    for v in values: 
        PHv = float(v[1])
        PPv = float(v[0])
        acumulador+= PHv * (PPi/PPv)
    acumulador = (alfa*acumulador) + (1-alfa)
    valor = list(valor)
    valor[-1] = acumulador
    context.write(key[0],valor)

alfa = 0.1
jobCountBoth = Job(inputDir, outputDir, fmap, fred)
jobCountBoth.setParams(alfa)
jobCountBoth.setShuffleCmp(cmpShort)
jobCountBoth.setSortCmp(cmpShuffle)
for i in range(20):
    success = jobCountBoth.waitForCompletion()

print(success)

True


Tercer Job saca top 10 

In [598]:
inputDir = "./format/"
outputDir = "./out/"

In [599]:
import heapq

def fmap(key, value, context):
    lista = value.split("\t")
    context.write(1, (key, float(lista[-1])))

def fcombiner(key, values, context):
    # simple combiner: sum counts locally
    top_maximo = []
    
    for v in values:
        elemento = (v[1],v[0])
        if len(top_maximo) < 10:
            # Si hay menos de 10 elementos, simplemente lo añadimos al heap.
            heapq.heappush(top_maximo, elemento)
        else:
            # Si el nuevo score (elemento[0]) es MAYOR que el elemento MÁS PEQUEÑO 
            # del heap (top_maximo[0]), reemplazamos el pequeño.
            if elemento[0] > top_maximo[0][0]:
                heapq.heapreplace(top_maximo, elemento)
    top_10_final = sorted(top_maximo, key=lambda e: e[0], reverse=True)
    for (maximo, clave) in top_10_final:
        context.write(key, (clave,maximo))

def fred(key, values, context):
    # simple combiner: sum counts locally
    top_maximo = []
    
    for v in values:
        elemento = (v[1],v[0])
        if len(top_maximo) < 10:
            # Si hay menos de 10 elementos, simplemente lo añadimos al heap.
            heapq.heappush(top_maximo, elemento)
        else:
            # Si el nuevo score (elemento[0]) es MAYOR que el elemento MÁS PEQUEÑO 
            # del heap (top_maximo[0]), reemplazamos el pequeño.
            if elemento[0] > top_maximo[0][0]:
                heapq.heapreplace(top_maximo, elemento)
    top_10_final = sorted(top_maximo, key=lambda e: e[0], reverse=True)
    for (maximo, clave) in top_10_final:
        context.write(clave, maximo)

jobCountBoth = Job(inputDir, outputDir, fmap, fred)
jobCountBoth.setCombiner(fcombiner)
success = jobCountBoth.waitForCompletion()

print(success)

True
